## Lesson 6: Essay Writer

https://learn.deeplearning.ai/courses/ai-agents-in-langgraph/lesson/7/essay-writer

In [23]:
from typing import List, TypedDict

from dotenv import find_dotenv, load_dotenv
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.graph import END, StateGraph

In [24]:
load_dotenv(find_dotenv())

MODEL = "glm-4-air"
BASE_URL = "https://open.bigmodel.cn/api/paas/v4/"

In [25]:
memory = SqliteSaver.from_conn_string(":memory:")

In [26]:
class AgentState(TypedDict):
    task: str
    plan: str
    draft: str
    critique: str
    content: List[str]
    revision_number: int
    max_revisions: int

In [27]:
model = ChatOpenAI(model=MODEL, temperature=0.01, base_url=BASE_URL)

In [28]:
PLAN_PROMPT = """You are an expert writer tasked with writing a high level outline of an essay. \
Write such an outline for the user provided topic. Give an outline of the essay along with any relevant notes \
or instructions for the sections."""


WRITER_PROMPT = """You are an essay assistant tasked with writing excellent 5-paragraph essays.\
Generate the best essay possible for the user's request and the initial outline. \
If the user provides critique, respond with a revised version of your previous attempts. \
Utilize all the information below as needed:\

------

{content}"""


REFLECTION_PROMPT = """You are a teacher grading an essay submission. \
Generate critique and recommendations for the user's submission. \
Provide detailed recommendations, including requests for length, depth, style, etc."""


RESEARCH_PLAN_PROMPT = """You are a researcher charged with providing information that can \
be used when writing the following essay. Generate a list of search queries that will gather \
any relevant information. Only generate 3 queries max."""


RESEARCH_CRITIQUE_PROMPT = """You are a researcher charged with providing information that can \
be used when making any requested revisions (as outlined below). \
Generate a list of search queries that will gather any relevant information. Only generate 3 queries max."""

In [29]:
from langchain_core.pydantic_v1 import BaseModel

In [30]:
class Queries(BaseModel):
    queries: List[str]

In [31]:
import os

from tavily import TavilyClient

In [32]:
tavily = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])

In [33]:
def plan_node(state: AgentState) -> dict:
    messages = [
        SystemMessage(content=PLAN_PROMPT),
        HumanMessage(content=state["task"]),
    ]
    response = model.invoke(messages)

    return {"plan": response.content}

In [34]:
def research_plan_node(state: AgentState) -> dict:
    queries = model.with_structured_output(Queries).invoke(
        [
            SystemMessage(content=RESEARCH_PLAN_PROMPT),
            HumanMessage(content=state["task"]),
        ]
    )

    content = state["content"] or []

    for q in queries.queries:
        response = tavily.search(query=q, max_results=2)

        for r in response["results"]:
            content.append(r["content"])

    return {"content": content}

In [35]:
def generation_node(state: AgentState) -> dict:
    content = "\n\n".join(state["content"] or [])

    user_message = HumanMessage(
        content=f"{state['task']}\n\nHere is my plan:\n\n{state['plan']}"
    )

    messages = [
        SystemMessage(content=WRITER_PROMPT.format(content=content)),
        user_message,
    ]

    response = model.invoke(messages)

    return {
        "draft": response.content,
        "revision_number": state.get("revision_number", 1) + 1,
    }

In [36]:
def reflection_node(state: AgentState) -> dict:
    messages = [
        SystemMessage(content=REFLECTION_PROMPT),
        HumanMessage(content=state["draft"]),
    ]

    response = model.invoke(messages)

    return {"critique": response.content}

In [37]:
def research_critique_node(state: AgentState) -> dict:
    queries = model.with_structured_output(Queries).invoke(
        [
            SystemMessage(content=RESEARCH_CRITIQUE_PROMPT),
            HumanMessage(content=state["critique"]),
        ]
    )

    content = state["content"] or []

    for q in queries.queries:
        response = tavily.search(query=q, max_results=2)

        for r in response["results"]:
            content.append(r["content"])

    return {"content": content}

In [38]:
def should_continue(state: AgentState) -> str:
    return END if state["revision_number"] >= state["max_revisions"] else "reflect"

In [39]:
builder = StateGraph(AgentState)

builder.add_node("planner", plan_node)
builder.add_node("generate", generation_node)
builder.add_node("reflect", reflection_node)
builder.add_node("research_plan", research_plan_node)
builder.add_node("research_critique", research_critique_node)

builder.set_entry_point("planner")

builder.add_conditional_edges(
    "generate", should_continue, {END: END, "reflect": "reflect"}
)

builder.add_edge("planner", "research_plan")
builder.add_edge("research_plan", "generate")
builder.add_edge("reflect", "research_critique")
builder.add_edge("research_critique", "generate")

graph = builder.compile(checkpointer=memory)

In [40]:
thread = {"configurable": {"thread_id": "1"}}

for s in graph.stream(
    {
        "task": "what is the difference between langchain and langsmith?",
        "max_revisions": 2,
        "revision_number": 1,
    },
    thread,
):
    print(s)

{'planner': {'plan': "**Essay Title:** The Distinctions Between LangChain and LangSmith: A Comparative Analysis\n\n**I. Introduction**\n   A. Brief overview of AI and natural language processing (NLP) advancements\n   B. Introduction to LangChain and LangSmith as AI-based NLP tools\n   C. Purpose of the essay: to explore and compare the differences between LangChain and LangSmith\n   D. Thesis statement: While LangChain and LangSmith are both designed to enhance NLP capabilities, they differ significantly in their architecture, functionality, and application scope.\n\n**II. Background and Overview of LangChain**\n   A. Definition and origin of LangChain\n   B. Core features and capabilities\n   C. Underlying technology and architecture\n   D. Key applications and use cases\n\n**III. Background and Overview of LangSmith**\n   A. Definition and origin of LangSmith\n   B. Core features and capabilities\n   C. Underlying technology and architecture\n   D. Key applications and use cases\n\n

**easy-to-read output:**

In [ ]:
{
    "planner": {
        "plan": """
        **Essay Title:** The Distinctions Between LangChain and LangSmith: A Comparative Analysis
            **I. Introduction**
                A. Brief overview of AI and natural language processing (NLP) advancements
                B. Introduction to LangChain and LangSmith as AI-based NLP tools
                C. Purpose of the essay: to explore and compare the differences between LangChain and LangSmith
                D. Thesis statement: While LangChain and LangSmith are both designed to enhance NLP capabilities, they differ significantly in their architecture, functionality, and application scope.
            **II. Background and Overview of LangChain**
                A. Definition and origin of LangChain
                B. Core features and capabilities
                C. Underlying technology and architecture
                D. Key applications and use cases
            **III. Background and Overview of LangSmith**
                A. Definition and origin of LangSmith
                B. Core features and capabilities
                C. Underlying technology and architecture
                D. Key applications and use cases
            **IV. Comparative Analysis**
                A. Architecture and Design
                    1. LangChain's modular approach
                    2. LangSmith's integrated framework
                B. Functionality and Performance
                    1. LangChain's strength in text generation and summarization
                    2. LangSmith's focus on language understanding and context retention
                C. Customization and Flexibility
                    1. LangChain's adaptability for various NLP tasks
                    2. LangSmith's specialized tools for specific NLP challenges
                D. Integration and Compatibility
                    1. LangChain's seamless integration with other AI tools
                    2. LangSmith's compatibility with existing systems and platforms
                E. Community and Support
                    1. LangChain's open-source community and resources
                    2. LangSmith's professional support and customer service
            **V. Use Cases and Impact**
                A. Real-world applications of LangChain
                    1. Examples in content creation, chatbots, and education
                B. Real-world applications of LangSmith
                    1. Examples in customer service, data analysis, and healthcare
                C. Comparative impact on industries and sectors
            **VI. Challenges and Limitations**
                A. Challenges faced by LangChain users
                    1. Scalability issues
                    2. Resource-intensive operations
                B. Challenges faced by LangSmith users
                    1. Limited customization options
                    2. Integration complexities
            **VII. Future Prospects and Trends**
                A. Future developments in LangChain technology
                B. Future developments in LangSmith technology
                C. Emerging trends in NLP and their implications for both tools
            **VIII. Conclusion**
                A. Recap of the key differences between LangChain and LangSmith
                B. Evaluation of their respective strengths and weaknesses
                C. Final thoughts on the evolving landscape of NLP tools and their role in AI
            **Notes and Instructions:**
                - Ensure that the essay maintains a neutral tone, providing an objective comparison between the two tools.
                - Use credible sources to support the analysis, including official documentation, academic papers, and industry reports.
                - Include examples and case studies to illustrate the practical applications and impact of each tool.
                - Address potential biases or limitations in the available research and data.
                - Conclude with a forward-looking perspective, considering the potential future developments and the broader implications for the field of NLP.
        """
    }
}

In [ ]:
{
    "research_plan": {
        "content": [
            "Find software to compare. Compare LangChain vs. LangSmith using this comparison chart. Compare price, features, and reviews of the software side-by-side to make the best choice for your business.",
            "Langsmith started charging. Time to compare alternatives. : r/LangChain. Langsmith started charging. Time to compare alternatives. Hey r/Langchain ! I've been using Langsmith for a while, and while it's been great, I'm curious about what else is out there. Specifically, I'm on the hunt for something fresh in the realm of LLM observability tools.",
            "Both types of tokens can be used to authenticate requests to the LangSmith API, but they have different use cases. Read more about the differences between Service Keys and Personal Access Tokens under admin concepts. Create an API key To log traces and run evaluations with LangSmith, you will need to create an API key to authenticate your requests.",
            "Unveiling Their Unique Roles in Language Models\nRead more\nLangChain vs. OpenAI: Which Unlocks Your App's True Potential?\nRead more\nWhich AI Reigns Supreme: LangChain vs. Hugging Face?\nRead more\nLangChain vs Haystack: Which Triumphs in LLM Orchestration?\nRead more\nLangChain vs. AutoGen: Which Framework Powers AI More Effectively?\nRead more\nSign in\nDon't have an account yet?\nSign up here\nSign up\nAlready have an account?\nSign in here\nForgot password?\nRemember your password?\nSign in here\nUnlock Your Career Potential with Our Online Courses: Explore Reviews, Roadmaps, Tutorials, and Learning Apps to Land Your Dream Job. Why Choose LangSmith for Production?\nLangSmith justifies its cost through its focus on five core pillars designed to enhance the development and management of LLM-powered applications:\nA significant advantage of using LangSmith is its user-friendly interface that lowers the entry barrier for individuals without extensive software development experience. LangSmith: Your Dashboard to LangChain's Engine\nJust as a car dashboard provides crucial information about the performance of a vehicle, LangSmith offers a comprehensive view into the workings of your language model applications powered by LangChain. In essence, while the cost of using LangSmith may be higher than LangChain, the investment is compensated by a significant reduction in long-term production risks and the assurance of delivering a polished, professional product.\n Comments\nDelete comment\nAre you sure you want to delete the comment?\nRead more\nChoosing AI Partners: LangChain's Innovation or Bedrock's Simplicity?\nRead more\nLangChain vs RAG:",
            "Reviewing the Results\nThe comparison views also make it easy to manually review the outputs to get a better sense for how the models behave, so you can make adjustments to your cognitive architecture and update the evaluation techniques to address any failure modes you identify.\n Both say that they cannot see an explicit answer to the question (since the retriever is the same), but the assistant is willing to provide context on how strict is used in other contexts as a pointer for the user.\n If you are using a model like gpt-3.5 and getting hallucinations like this, you can try adding an additional system prompt reminding the model to only respond based on the retrieved content, and you can work to improve the retriever to filter out irrelevant documents.\n If you compare the outputs, you can see that mistral was influenced too much by the document content and mentioned the class from the docs rather than the direct answer.\n As a part of the initial release, we have evaluated various implementations that differ across a few dimensions:\nYou can check out the link above to review the results or continue with the information below!\n",
            "Langchain vs Langsmith: Unpacking the AI Language Model Showdown\nOverview of Langchain and Langsmith\nLangchain is a versatile open-source framework that enables you to build applications utilizing large language models (LLM) like GPT-3. Check out our free WhatsApp channel to stay educated on LLM developments:\nJoin the Finxter Academy and unlock access to premium courses 👑 to certify your skills in exponential technologies and programming.\n Frequently Asked Questions\nWhether you’re trying to figure out which tool fits your needs or you’re just getting started with language model automation, these FAQs will help shed light on the common curiosities about Langchain and LangSmith.\n The best way to find out is to reach out to them through the LangSmith Walkthrough page or to inquire about access directly through their support channels.\n Here’s how you might start a simple Langchain project in Python:\nTo integrate LangSmith, you could write something like this:\nYou’re not limited to Python, though.",
        ]
    }
}

In [ ]:
{
    "generate": {
        "draft": """
            **The Distinctions Between LangChain and LangSmith: A Comparative Analysis**
                **I. Introduction**
                    In the rapidly evolving landscape of artificial intelligence, natural language processing (NLP) has seen significant advancements, enabling machines to understand, interpret, and generate human language. Among the myriad of tools that have emerged, LangChain and LangSmith stand out as powerful AI-based NLP tools. This essay aims to explore and compare the differences between these two tools, providing insights into their unique offerings. The thesis of this essay is that while LangChain and LangSmith are both designed to enhance NLP capabilities, they differ significantly in their architecture, functionality, and application scope.
                **II. Background and Overview of LangChain**
                    LangChain is an open-source framework that empowers developers to build applications leveraging large language models (LLMs) such as GPT-3. Its origin is rooted in the need for a flexible and versatile platform that can adapt to a wide range of NLP tasks. LangChain's core features include robust text generation, summarization, and the ability to perform complex language tasks. Its architecture is modular, allowing for easy integration with other AI tools and frameworks. Key applications of LangChain span content creation, chatbots, and educational platforms, where its strength in generating coherent and contextually relevant text is highly valued.
                **III. Background and Overview of LangSmith**
                    LangSmith, on the other hand, is a tool that has evolved with a focus on providing a comprehensive view into the workings of language model applications. It emerged as a solution for those seeking an integrated framework that simplifies the development and management of LLM-powered applications. LangSmith's core features are centered around language understanding and context retention, making it particularly useful for tasks that require deep comprehension of the input text. Its architecture is designed to be user-friendly, lowering the entry barrier for individuals without extensive software development experience. LangSmith finds its niche in customer service, data analysis, and healthcare, where its ability to provide detailed insights and maintain context is crucial.
                **IV. Comparative Analysis**
                    When comparing the two tools, several key differences emerge:
                    A. **Architecture and Design**: LangChain's modular approach allows for greater flexibility and customization, enabling developers to tailor the tool to their specific needs. LangSmith, conversely, offers an integrated framework that streamlines the development process and provides a comprehensive dashboard for monitoring and managing language model applications.
                    B. **Functionality and Performance**: LangChain excels in text generation and summarization, making it ideal for applications that require the creation of human-like text. LangSmith, however, shines in language understanding and context retention, which is essential for applications that demand a deep understanding of the input text.
                    C. **Customization and Flexibility**: LangChain's adaptability makes it suitable for a wide array of NLP tasks, from simple text processing to complex language understanding. LangSmith, while less customizable, provides specialized tools that are optimized for specific NLP challenges.
                    D. **Integration and Compatibility**: LangChain's seamless integration with other AI tools makes it a versatile choice for developers who need to connect multiple services. LangSmith, designed for compatibility with existing systems, ensures that businesses can incorporate the tool without disrupting their current operations.
                    E. **Community and Support**: LangChain benefits from a vibrant open-source community that provides a wealth of resources and support. LangSmith, on the other hand, offers professional support and customer service, which can be particularly valuable for businesses that require dedicated assistance.
                **V. Use Cases and Impact**
                    The real-world applications of LangChain and LangSmith highlight their respective strengths. LangChain has been utilized in content creation to generate articles and reports, in chatbots for customer interaction, and in education to provide automated tutoring. LangSmith, with its focus on understanding and context, is applied in customer service to enhance interaction quality, in data analysis to derive meaningful insights, and in healthcare to assist with patient information management. The comparative impact on industries and sectors underscores the importance of choosing the right tool for the specific needs of an application.
                **VI. Challenges and Limitations**
                    Despite their strengths, both LangChain and LangSmith face challenges. LangChain users may encounter scalability issues and resource-intensive operations, which can be a barrier for smaller-scale applications. LangSmith users, while benefiting from a user-friendly interface, may find the tool's customization options limited and may face complexities during integration with certain systems.
                **VII. Future Prospects and Trends**
                    Looking forward, LangChain is likely to continue evolving with developments in its technology, potentially improving scalability and reducing resource demands. LangSmith may focus on expanding its customization features and enhancing its compatibility with a wider range of platforms. Emerging trends in NLP, such as increased personalization and greater context-awareness, will undoubtedly influence the future developments of both tools.
                **VIII. Conclusion**
                    In conclusion, LangChain and LangSmith offer distinct advantages in the realm of NLP, each with its own set of strengths and weaknesses. LangChain's modular architecture and text generation capabilities make it a powerful choice for a broad
        """,
        "revision_number": 2,
    }
}